In [1]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.preprocessing import StandardScaler
from scipy import stats as s
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
import sklearn.cross_validation as crv
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
import random
from sklearn.metrics import mean_squared_error

/Users/takeshin/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/Users/takeshin/anaconda3/lib/python3.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def col_name():
    p0 = ['x','y','z']
    marker =[]
    p2 =[]
    for a in range(1,44):       #Mocapマーカー数:43個
        p1='m'+ str(a)
        for b in p0:
            p2 = p1 + b
            marker.append(p2)
    return marker

def read_mocap(file):
    data_mocap = pd.read_csv(file,sep=' ',header=None)
    #座標値列の列名変更
    data_mocap=data_mocap.rename(columns={data_mocap.columns[a]:col_name()[a] for a in range(129)} )
    #座標値列以外の列名変更
    data_mocap=data_mocap.rename(columns={129:'FrameNumber',130:'TimeStamp'})
    return data_mocap

#fix NaN
def fix_NaN(data):
    for  c  in  data.columns:
        col = data[c]  #'X'列から1列ずつ見る
        for r in range(len(data)):  #1行ずつ見る
            if col[r] == 0.0:       #0値が見つかれば
                data.iloc[r][c] = 'NaN' #'NaN'を代入
    mc_clean = data.interpolate()
    mc_clean = mc_clean.fillna(0)
    return mc_clean

def obtain_statistical_feature(window):
    #features = np.array(np.array(np.var(window, axis=0)))
    #features=np.array(np.array(np.mean(window-np.mean(window), axis=0)))
    features=np.array( np.array(np.mean(window-np.median(window), axis=0))) 
    features=np.append(features, np.array(np.std(window, axis=0))) 
    features=np.append(features, np.array(np.var(window, axis=0)))
    features=np.append(features, np.array(s.skew(window, axis=0)))
    features=np.append(features, np.array(s.kurtosis(window, axis=0)))
    features=np.append(features, np.array(get_tw_col_var(window)))
    return features.reshape(1,len(features))


def get_tw_variance(window):
    total_cols = window.shape[1]
    tw_var = []
    for i in range (total_cols):
        column = window.iloc[:,i]
        tw_var.append(get_tw_col_var(column))
    return tw_var


def get_tw_col_var(column):
    mean = np.mean(column)
    total = len(column)
    
    v_sum = 0
    for i in range(total):
        w = np.exp(-0.5*(total-i))
        #print("el peso", w, "indice", i, "total", total)
        v = w*np.square(column[i]-mean)
        v_sum += v
    return v_sum

#速度加速度を求める関数を定義
def cal_va(data):
    diffs = pd.DataFrame()
    for i in range(len(data)-1):
        diff = data.iloc[i+1]-data.iloc[i]
        diffs = diffs.append(diff,ignore_index=True)
        
    vs = pd.DataFrame()
    for i in range(len(diffs)):
        v = diffs.iloc[i][1:]/diffs.iloc[i]["TimeStamp"]
        vs = vs.append(v,ignore_index=True)
    return vs

In [3]:
def read_acc(file):
    data_acc = pd.read_csv(file,sep=' ',header=None)
   
    data_acc = data_acc.rename(columns={data_acc.columns[0]:'X_acc', data_acc.columns[1]:'Y_acc', data_acc.columns[2]:'Z_acc',data_acc.columns[3]:'TimeStamp'} )
    
    return data_acc



#線形加速度を得る
def linear_acc(data):
    alpha = 0.9
    for  c  in  ['X_acc','Y_acc','Z_acc']:
        g = 0  
        col = data[c]           #'X'列から1列ずつ見る
        for r in range(len(data)):         #1行ずつ見る
            #Isolate the force of gravity with the low-pass filter.
            g = alpha*g + (1-alpha) * col[r]
            #Remove the gravity contribution with the high-pass filter.
            linear = col[r] - g
            data.iloc[r][c] = linear
        linear_acc = data
    return linear_acc

def my_index_multi(l, x):
    return [i for i, _x in enumerate(l) if _x == x]

In [4]:
def one_acc1(data):
    xyz = pd.DataFrame(columns=["acc"])
    accs = []
    for i in range(len(data)):
        #ベクトルの大きさを求める
        acc = np.linalg.norm(data.loc[i])
        accs.append(acc)
    xyz["acc"]= accs
    #xyz["TimeStamp"] = data["TimeStamp"].tolist()
    return xyz

In [5]:
def one_acc2(data):
    xyz = pd.DataFrame(columns=["acc"])
    accs = []
    for i in range(len(data)):
        #ベクトルの大きさを求める
        acc = np.linalg.norm(data.loc[i][data.columns != "TimeStamp"])
        accs.append(acc)
    xyz["acc"]= accs
    #xyz["TimeStamp"] = data["TimeStamp"].tolist()
    return xyz

In [6]:
def main_functuion(sampling_rate,marker,sensor):
    x = np.empty([0,1])
    y = np.empty([0,])
    random.seed(0)
    print("marker:",marker)
    print("sensor:",sensor)
    #Read all mocap data and acc data
    #subjects =  ["01","02","03","04","05","06","07","08","09","10","11","12"]
    subjects = random.sample( ["01","02","03","04","05","06","07","08","09","10","11","12"],8)
    actions = ["01","02","03","04","05","06","07","08","09","10","11"]
   
    for subject in subjects:
        for action in actions:
            for record in range(1,6):
            #for record in random.sample( [1,2,3,4,5],2): 
                try:
                    data = read_mocap('/Users/takeshin/odrive/sozolab_gdrive/students/Shingo/BerkeleyMHAD/Mocap/OpticalData/moc_s'+str(subject)+'_a'+str(action)+'_r0'+str(record)+'.txt')
                    print('moc_s'+str(subject)+'_a'+str(action)+'_r0'+str(record)+'.txt')
                    columns = ['m'+str(marker)+'x','m'+str(marker)+'y','m'+str(marker)+'z']
                    cols = np.array(columns)
                    cols = list(np.ravel(cols))
                    cols.append("TimeStamp")
                    data = data[cols]
                    data = fix_NaN(data)
                    
                    i = 0
                    mc_ds = pd.DataFrame()
                    #処理
                    while  i < (len(data)//(480//sampling_rate))*(480//sampling_rate):
                        mc_ds = mc_ds.append(data.iloc[i : i+(480//sampling_rate),:].mean(),ignore_index=True)
                        i+=(480//sampling_rate)
                    if i < len(data):
                        mc_ds = mc_ds.append(data.iloc[i :,:].mean(),ignore_index=True)
                    
                    
                    v = cal_va(mc_ds)
                    times = mc_ds['TimeStamp'].shift(periods=-1)
                    mc_v = pd.DataFrame(v)
                    mc_v["TimeStamp"]=times
                    mc_v = mc_v.dropna()

                    
                    a = cal_va(mc_v)
                    times = mc_v['TimeStamp'].shift(periods=-1)
                    mc_a = pd.DataFrame(a)
                    mc_a["TimeStamp"]=times
                    mc_a = mc_a.dropna()
                    #print(mc_a)                                           
                
                    
                    #単位を[G]に変換  [mm/s^2] →  [G]
                    accel = mc_a.loc[:, mc_a.columns != 'TimeStamp']*0.001/9.80665
                    accel = accel.rolling(window= 10, center=False).mean()
                    accel = accel.dropna()
                    accel = accel.reset_index()
                    #print(accel)
                    v_a = one_acc1(accel)
                    #print(v_a)
                    
                   
                    

                    #リアル加速度データの読み込み
                    r_a = read_acc('/Users/takeshin/odrive/sozolab_gdrive/students/Shingo/BerkeleyMHAD/Accelerometer/Shimmer0'+str(sensor)+'/acc_h0'+str(sensor)+'_s'+str(subject)+'_a'+str(action)+'_r0'+str(record)+'.txt')
                    print('acc_h0'+str(sensor)+'_s'+str(subject)+'_a'+str(action)+'_r0'+str(record)+'.txt')
                    r_a = fix_NaN(r_a)
                    r_a = linear_acc(r_a) 
                    r_a = one_acc2(r_a)
                    #print(r_a)
                    

                    i = 0
                    #処理
                    if len(v_a) < len(r_a):
                        x =np.append(x,v_a, axis=0)
                        y = np.append(y,r_a["acc"][0:len(v_a)], axis=0)
                    else:
                        x =np.append(x,v_a[0:len(r_a)], axis=0)
                        y = np.append(y,r_a["acc"][0:len(r_a)], axis=0)
                        
                    
                except FileNotFoundError:
                    pass
                    
    return x, y

In [7]:
x,y = main_functuion(30,30,3)   

marker: 30
sensor: 3
moc_s07_a01_r01.txt
acc_h03_s07_a01_r01.txt
moc_s07_a01_r02.txt
acc_h03_s07_a01_r02.txt
moc_s07_a01_r03.txt
acc_h03_s07_a01_r03.txt
moc_s07_a01_r04.txt
acc_h03_s07_a01_r04.txt
moc_s07_a01_r05.txt
acc_h03_s07_a01_r05.txt
moc_s07_a02_r01.txt
acc_h03_s07_a02_r01.txt
moc_s07_a02_r02.txt
acc_h03_s07_a02_r02.txt
moc_s07_a02_r03.txt
acc_h03_s07_a02_r03.txt
moc_s07_a02_r04.txt
acc_h03_s07_a02_r04.txt
moc_s07_a02_r05.txt
acc_h03_s07_a02_r05.txt
moc_s07_a03_r01.txt
acc_h03_s07_a03_r01.txt
moc_s07_a03_r02.txt
acc_h03_s07_a03_r02.txt
moc_s07_a03_r03.txt
acc_h03_s07_a03_r03.txt
moc_s07_a03_r04.txt
acc_h03_s07_a03_r04.txt
moc_s07_a03_r05.txt
acc_h03_s07_a03_r05.txt
moc_s07_a04_r01.txt
acc_h03_s07_a04_r01.txt
moc_s07_a04_r02.txt
acc_h03_s07_a04_r02.txt
moc_s07_a04_r03.txt
acc_h03_s07_a04_r03.txt
moc_s07_a04_r04.txt
acc_h03_s07_a04_r04.txt
moc_s07_a04_r05.txt
acc_h03_s07_a04_r05.txt
moc_s07_a05_r01.txt
acc_h03_s07_a05_r01.txt
moc_s07_a05_r02.txt
acc_h03_s07_a05_r02.txt
moc_s07_a05

moc_s05_a05_r02.txt
acc_h03_s05_a05_r02.txt
moc_s05_a05_r03.txt
acc_h03_s05_a05_r03.txt
moc_s05_a05_r04.txt
acc_h03_s05_a05_r04.txt
moc_s05_a05_r05.txt
acc_h03_s05_a05_r05.txt
moc_s05_a06_r01.txt
acc_h03_s05_a06_r01.txt
moc_s05_a06_r02.txt
acc_h03_s05_a06_r02.txt
moc_s05_a06_r03.txt
acc_h03_s05_a06_r03.txt
moc_s05_a06_r04.txt
acc_h03_s05_a06_r04.txt
moc_s05_a06_r05.txt
acc_h03_s05_a06_r05.txt
moc_s05_a07_r01.txt
acc_h03_s05_a07_r01.txt
moc_s05_a07_r02.txt
acc_h03_s05_a07_r02.txt
moc_s05_a07_r03.txt
acc_h03_s05_a07_r03.txt
moc_s05_a07_r04.txt
acc_h03_s05_a07_r04.txt
moc_s05_a07_r05.txt
acc_h03_s05_a07_r05.txt
moc_s05_a08_r01.txt
acc_h03_s05_a08_r01.txt
moc_s05_a08_r02.txt
acc_h03_s05_a08_r02.txt
moc_s05_a08_r03.txt
acc_h03_s05_a08_r03.txt
moc_s05_a08_r04.txt
acc_h03_s05_a08_r04.txt
moc_s05_a08_r05.txt
acc_h03_s05_a08_r05.txt
moc_s05_a09_r01.txt
acc_h03_s05_a09_r01.txt
moc_s05_a09_r02.txt
acc_h03_s05_a09_r02.txt
moc_s05_a09_r03.txt
acc_h03_s05_a09_r03.txt
moc_s05_a09_r04.txt
acc_h03_s05_

acc_h03_s03_a09_r04.txt
moc_s03_a09_r05.txt
acc_h03_s03_a09_r05.txt
moc_s03_a10_r01.txt
acc_h03_s03_a10_r01.txt
moc_s03_a10_r02.txt
acc_h03_s03_a10_r02.txt
moc_s03_a10_r03.txt
acc_h03_s03_a10_r03.txt
moc_s03_a10_r04.txt
acc_h03_s03_a10_r04.txt
moc_s03_a10_r05.txt
acc_h03_s03_a10_r05.txt
moc_s03_a11_r01.txt
acc_h03_s03_a11_r01.txt
moc_s03_a11_r02.txt
acc_h03_s03_a11_r02.txt
moc_s03_a11_r03.txt
acc_h03_s03_a11_r03.txt
moc_s03_a11_r04.txt
acc_h03_s03_a11_r04.txt
moc_s03_a11_r05.txt
acc_h03_s03_a11_r05.txt
moc_s11_a01_r01.txt
acc_h03_s11_a01_r01.txt
moc_s11_a01_r02.txt
acc_h03_s11_a01_r02.txt
moc_s11_a01_r03.txt
acc_h03_s11_a01_r03.txt
moc_s11_a01_r04.txt
acc_h03_s11_a01_r04.txt
moc_s11_a01_r05.txt
acc_h03_s11_a01_r05.txt
moc_s11_a02_r01.txt
acc_h03_s11_a02_r01.txt
moc_s11_a02_r02.txt
acc_h03_s11_a02_r02.txt
moc_s11_a02_r03.txt
acc_h03_s11_a02_r03.txt
moc_s11_a02_r04.txt
acc_h03_s11_a02_r04.txt
moc_s11_a02_r05.txt
acc_h03_s11_a02_r05.txt
moc_s11_a03_r01.txt
acc_h03_s11_a03_r01.txt
moc_s11_

In [8]:
print(x.shape)
print(y.shape)


(89975, 1)
(89975,)


In [9]:
#変換モデル 回帰
clf2 = RandomForestRegressor()
#clf2 = LinearRegression()
clf2.fit(x, y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [10]:
def main_functuion2(sampling_rate,*markers):
    X = np.empty([0,6*len(markers)])
    y = np.empty([0,])
    random.seed(0)
    #Read all mocap data
    b = random.sample( ["01","02","03","04","05","06","07","08","09","10","11","12"],8)
    a =  ["01","02","03","04","05","06","07","08","09","10","11","12"]
    subjects = list(set(a) - set(b))
    #subjects =  ["01","02","03","04","05","06","07","08","09","10","11","12"]
    actions = ["01","02","03","04","05","06","07","08","09","10","11"]
    for subject in subjects:
        for action in actions:
            for record in range(1,6):
                try:
                    data = read_mocap('/Users/takeshin/odrive/sozolab_gdrive/students/Shingo/BerkeleyMHAD/Mocap/OpticalData/moc_s'+str(subject)+'_a'+str(action)+'_r0'+str(record)+'.txt')
                    print('moc_s'+str(subject)+'_a'+str(action)+'_r0'+str(record)+'.txt')
                    columns = [['m'+str(i)+'x','m'+str(i)+'y','m'+str(i)+'z'] for i in markers]
                    cols = np.array(columns)
                    cols = list(np.ravel(cols))
                    cols.append("TimeStamp")
                    data = data[cols]
                    data = fix_NaN(data)
                    
                    i = 0
                    mc_ds = pd.DataFrame()
                    #処理
                    while  i < (len(data)//(480//sampling_rate))*(480//sampling_rate):
                        mc_ds = mc_ds.append(data.iloc[i : i+(480//sampling_rate),:].mean(),ignore_index=True)
                        i+=(480//sampling_rate)
                    if i < len(data):
                        mc_ds = mc_ds.append(data.iloc[i :,:].mean(),ignore_index=True)
                
                    
                    v = cal_va(mc_ds)
                    times = mc_ds['TimeStamp'].shift(periods=-1)
                    mc_v = pd.DataFrame(v)
                    mc_v["TimeStamp"]=times
                    mc_v = mc_v.dropna()

                    
                    a = cal_va(mc_v)
                    times = mc_v['TimeStamp'].shift(periods=-1)
                    mc_a = pd.DataFrame(a)
                    mc_a["TimeStamp"]=times
                    mc_a = mc_a.dropna()
                    #print(mc_a)                                           
                
                    
                    #単位を[G]に変換  [mm/s^2] →  [G]
                    accel = mc_a.loc[:, mc_a.columns != 'TimeStamp']*0.001/9.80665
                    accel = accel.rolling(window= 10, center=False).mean()
                    accel = accel.dropna()
                    accel = accel.reset_index()
                    #print(accel)
                    v_a = one_acc1(accel)
                    #print(v_a)
                    
                    a_predict = clf2.predict(v_a).tolist()
                    #print(a_predict)
                    
                    sf = obtain_statistical_feature(a_predict)
                    X =np.append(X,sf, axis=0)
                    y = np.append(y,[action], axis=0)
                
                
                except FileNotFoundError:
                    pass
                    
    return X, y

In [11]:
x2,y2= main_functuion2(30,30)

moc_s10_a01_r01.txt
moc_s10_a01_r02.txt
moc_s10_a01_r03.txt
moc_s10_a01_r04.txt
moc_s10_a01_r05.txt
moc_s10_a02_r01.txt
moc_s10_a02_r02.txt
moc_s10_a02_r03.txt
moc_s10_a02_r04.txt
moc_s10_a02_r05.txt
moc_s10_a03_r01.txt
moc_s10_a03_r02.txt
moc_s10_a03_r03.txt
moc_s10_a03_r04.txt
moc_s10_a03_r05.txt
moc_s10_a04_r01.txt
moc_s10_a04_r02.txt
moc_s10_a04_r03.txt
moc_s10_a04_r04.txt
moc_s10_a04_r05.txt
moc_s10_a05_r01.txt
moc_s10_a05_r02.txt
moc_s10_a05_r03.txt
moc_s10_a05_r04.txt
moc_s10_a05_r05.txt
moc_s10_a06_r01.txt
moc_s10_a06_r02.txt
moc_s10_a06_r03.txt
moc_s10_a06_r04.txt
moc_s10_a06_r05.txt
moc_s10_a07_r01.txt
moc_s10_a07_r02.txt
moc_s10_a07_r03.txt
moc_s10_a07_r04.txt
moc_s10_a07_r05.txt
moc_s10_a08_r01.txt
moc_s10_a08_r02.txt
moc_s10_a08_r03.txt
moc_s10_a08_r04.txt
moc_s10_a08_r05.txt
moc_s10_a09_r01.txt
moc_s10_a09_r02.txt
moc_s10_a09_r03.txt
moc_s10_a09_r04.txt
moc_s10_a09_r05.txt
moc_s10_a10_r01.txt
moc_s10_a10_r02.txt
moc_s10_a10_r03.txt
moc_s10_a10_r04.txt
moc_s10_a10_r05.txt


In [12]:
print(x2.shape)
print(y2.shape)

(220, 6)
(220,)


In [13]:
x2

array([[-0.01397119,  0.41310226,  0.17065348,  0.32997434, -0.49074166,
         0.58039883],
       [ 0.08607218,  0.40733252,  0.16591978,  0.81139049,  0.5022324 ,
         0.52380112],
       [ 0.03196544,  0.40242226,  0.16194368,  0.5008909 , -0.31890375,
         0.60909037],
       ...,
       [ 0.11177755,  0.31801316,  0.10113237,  2.07611866,  5.11966392,
         0.06745889],
       [ 0.06113122,  0.18329646,  0.03359759,  0.79104462, -0.41746798,
         0.06444878],
       [ 0.05004123,  0.15055481,  0.02266675,  1.62312013,  1.87823816,
         0.00852686]])

In [14]:
y2

array(['01', '01', '01', '01', '01', '02', '02', '02', '02', '02', '03',
       '03', '03', '03', '03', '04', '04', '04', '04', '04', '05', '05',
       '05', '05', '05', '06', '06', '06', '06', '06', '07', '07', '07',
       '07', '07', '08', '08', '08', '08', '08', '09', '09', '09', '09',
       '09', '10', '10', '10', '10', '10', '11', '11', '11', '11', '11',
       '01', '01', '01', '01', '01', '02', '02', '02', '02', '02', '03',
       '03', '03', '03', '03', '04', '04', '04', '04', '04', '05', '05',
       '05', '05', '05', '06', '06', '06', '06', '06', '07', '07', '07',
       '07', '07', '08', '08', '08', '08', '08', '09', '09', '09', '09',
       '09', '10', '10', '10', '10', '10', '11', '11', '11', '11', '11',
       '01', '01', '01', '01', '01', '02', '02', '02', '02', '02', '03',
       '03', '03', '03', '03', '04', '04', '04', '04', '04', '05', '05',
       '05', '05', '05', '06', '06', '06', '06', '06', '07', '07', '07',
       '07', '07', '08', '08', '08', '08', '08', '0

In [15]:
m1 = x2[my_index_multi(y2, '01')].mean()
m2 = x2[my_index_multi(y2, '02')].mean()
m3 = x2[my_index_multi(y2, '03')].mean()
m4 = x2[my_index_multi(y2, '04')].mean()
m5 = x2[my_index_multi(y2, '05')].mean()
m6 = x2[my_index_multi(y2, '06')].mean()
m7 = x2[my_index_multi(y2, '07')].mean()
m8 = x2[my_index_multi(y2, '08')].mean()
m9 = x2[my_index_multi(y2, '09')].mean()
m10 = x2[my_index_multi(y2, '10')].mean()
m11 = x2[my_index_multi(y2, '11')].mean()
print(m1)
print(m2)
print(m3)
print(m4)
print(m5)
print(m6)
print(m7)
print(m8)
print(m9)
print(m10)
print(m11)

0.233963144049374
0.22239933945166795
3.316436449662292
2.0660722019556657
4.411474964858553
3.961939821863002
2.295940552221026
0.7915749944205879
3.259375693128415
0.9639071724019814
0.7368511915100971


In [16]:
np.sqrt(((m1- 0.2837427766466512)**2+(m2- 0.1733721261595955)**2+(m3- 0.2416587854446975)**2//
+(m4- 1.7662228027076237)**2+(m5-5.252416738551928)**2+(m6-5.506056486778035)**2//
+(m7-2.3397725028515692)**2+(m8-1.1361369336776836)**2+(m9-1.1488882440431472)**2//
 +(m10-0.8139112866822661)**2+(m11-0.6847317601151889)**2)/11)

11.846877241440312

In [12]:
def ml_ar(X,y):
    x_train, x_test, y_train, y_test = crv.train_test_split(X, y, test_size=0.30, random_state=42)
    clf = RandomForestClassifier(n_estimators=20, random_state=42)
    clf.fit(x_train, y_train)
    y_predict = clf.predict(x_test)
    conf = confusion_matrix(y_test, y_predict)
    print (accuracy_score(y_test, y_predict) ) 
    actions = ["01","02","03","04","05","06","07","08","09","10","11"]
    print(classification_report(y_test, y_predict, target_names=actions))
    print(conf)

In [13]:
ml_ar(x2,y2)

0.5555555555555556
             precision    recall  f1-score   support

         01       0.65      0.59      0.62        22
         02       0.61      0.61      0.61        18
         03       0.57      0.80      0.67        20
         04       0.54      0.39      0.45        18
         05       0.70      0.70      0.70        20
         06       0.38      0.24      0.29        21
         07       0.75      1.00      0.86        15
         08       0.44      0.47      0.45        15
         09       0.30      0.23      0.26        13
         10       0.53      0.47      0.50        19
         11       0.43      0.59      0.50        17

avg / total       0.54      0.56      0.54       198

[[13  7  0  0  0  1  0  0  0  0  1]
 [ 7 11  0  0  0  0  0  0  0  0  0]
 [ 0  0 16  0  0  0  0  0  4  0  0]
 [ 0  0  3  7  0  1  1  2  2  1  1]
 [ 0  0  0  0 14  5  1  0  0  0  0]
 [ 0  0  3  2  6  5  3  1  1  0  0]
 [ 0  0  0  0  0  0 15  0  0  0  0]
 [ 0  0  0  1  0  0  0  7  0  2  5]
 